**Problem Statement:**

In this, we have to preprocess raw text for next stage usage.

**Import necessary libraries**

In [1]:
#Load the libraries
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
import spacy
import re, string, unicodedata
from tqdm import tqdm 
from spacy.lang.en.stop_words import STOP_WORDS
import os
import warnings
from text_prep_utils.text_prep import Text_processing_pipeline
nlp = spacy.load('en_core_web_trf')

warnings.filterwarnings('ignore')

**Import the training dataset**

In [2]:
#importing the training data
# imdb_data=pd.read_csv('data/train.csv')
imdb_data=pd.read_csv('data/train.csv')
print(imdb_data.shape)
imdb_data.head(10)

(29341, 3)


,ID,review,sentiment
0,41411,I watched this film because I'm a big fan of R...,0
1,37586,It does not seem that this movie managed to pl...,1
2,6017,"Enough is not a bad movie , just mediocre .",0
3,44656,my friend and i rented this one a few nights a...,0
4,38711,"Just about everything in this movie is wrong, ...",0
5,24198,this isn't 'Bonnie and Clyde' or 'Thelma and L...,1
6,37654,I have to say that I really liked UNDER SIEGE ...,0
7,45340,Kramer Vs. Kramer is a near-heartening drama a...,1
8,9745,"Like the other comments says, this might be su...",0
9,55813,The tunes are the best aspect of this televisi...,1


In [3]:
#importing the testing data
# imdb_test_data=pd.read_csv('data/test.csv')
imdb_test_data=pd.read_csv('data/test.csv')
print(imdb_test_data.shape)
imdb_test_data.head(10)

(29341, 2)


,ID,review
0,22622,Robert Lansing plays a scientist experimenting...
1,10162,"Well I've enjoy this movie, even though someti..."
2,17468,First things first - though I believe Joel Sch...
3,42579,I watched this movie on the grounds that Amber...
4,701,A certain sexiness underlines even the dullest...
5,13418,"Why did they change the cute, Rugrats televisi..."
6,50674,Who will love my children has changed my heart...
7,33704,Im gonna make this short and sweet because i d...
8,21242,Though I never like to be the sort of person w...
9,40345,The main aspect about the Superstar's movies a...


**Exploratery data analysis**

In [4]:
#Summary of the dataset
imdb_data.describe()

,ID,sentiment
count,29341.000000,29341.000000
mean,29348.411097,0.509662
std,17002.074346,0.499915
min,4.000000,0.000000
25%,14564.000000,0.000000
50%,29348.000000,1.000000
75%,44162.000000,1.000000
max,58681.000000,1.000000


**Sentiment count**

In [5]:
#sentiment count
imdb_data['sentiment'].value_counts()

1    14954
0    14387
Name: sentiment, dtype: int64

We can see that the dataset is balanced.

**Spliting the training dataset**

In [6]:
#split the dataset  
#train dataset
train_reviews=imdb_data.review
train_sentiments=imdb_data.sentiment
#test dataset
test_reviews=imdb_test_data.review
# test_sentiments=imdb_test_data.sentiment
print(train_reviews.shape,train_sentiments.shape)
# print(test_reviews.shape,test_sentiments.shape)

(29341,) (29341,)


**Preliminary Text Preprocessing**

In [7]:
print(imdb_data.loc[7,'review'])

Kramer Vs. Kramer is a near-heartening drama about shocking, drastic augmentations of the two subjects of a failed married couple. Meryl Streep, in the throes of her trademark maternal sensitivity, plays an unhappy stay-at-home mother who feels confined to such a role and within the first five minutes of the film leaves her inattentive husband, in a fantastic performance by Dustin Hoffman, to find another role for herself. Hoffman is dumbstruck, having absolutely no idea what to do with himself, having taken so much for granted that he doesn't know the first thing about getting his son to school in the morning.<br /><br />Hoffman seamlessly characterizes this husband as such a juicy load of setbacks. He is restless, relentless and impatient, but even though the positive side to those three adjectives should include just the opposite, he is unremittingly fixated on whatever he turns his head to. He's been focused on his career in advertising, and when he is left to raise his son Billy a

In [8]:
"""Usage Example"""
nlp = spacy.load('en_core_web_trf')

Text_processing_pipeline(imdb_data.loc[7,'review']).main_pipeline()

'kramer vs kramer near heartening drama shocking drastic augmentation subject fail married couple meryl streep throe trademark maternal sensitivity play unhappy stay home mother feel confine role minute film leave inattentive husband fantastic performance dustin hoffman find role hoffman dumbstruck have absolutely idea having take grant doesn know thing get son school morning hoffman seamlessly characterize husband juicy load setback restless relentless impatient positive adjective include opposite unremittingly fixated turn head s focus career advertising leave raise son billy chaos usher immediately s throw temper tantrum quit angrily halfway activity awhile befriend neighbor joanna s friend play sexy jane alexander hoffman cool jet understand wife leave meantime boundless energy redirect raise billy lose job custody battle title brilliantly grey circumstance ending little unmotivated subjectified audience line shot witty screen writing touch diminish magical 1970s'

In [9]:
%%time
processed_text = []

for text in tqdm(imdb_data['review']):
    tmp = Text_processing_pipeline(text).main_pipeline()
    processed_text.append(tmp)

imdb_data['processed_review'] = processed_text

# or use multiprocessing to accelrate
# def mp_wrapper(text):
#     return text_processing_pipeline(text).main_pipeline()

# processed_text = list(map(mp_wrapper, imdb_data['review']))
# mp_wrapper(imdb_data.loc[7,'review'])
# processed_text = list(tqdm(pool.imap(mp_wrapper, imdb_data['review']), total=len(imdb_data['review'])))

# processed_text = list(tqdm(pool.imap(mp_wrapper, list(imdb_data.loc[0:3,'review'])), total=len(list(imdb_data.loc[0:3,'review']))))

  0%|          | 73/29341 [02:25<19:03:32,  2.34s/it]

In [16]:
# imdb_data.to_csv('./data/processed_train.csv',index=0)
imdb_data

,ID,review,sentiment,processed_review
0,41411,I watched this film because I'm a big fan of R...,0,watch film m big fan river phoenix joaquin pho...
1,37586,It does not seem that this movie managed to pl...,1,movie manage lot people see place bump acciden...
2,6017,"Enough is not a bad movie , just mediocre .",0,bad movie mediocre
3,44656,my friend and i rented this one a few nights a...,0,friend rent night ago single good movie see me...
4,38711,"Just about everything in this movie is wrong, ...",0,movie wrong wrong wrong mike myers example s r...
...,...,...,...,...
29336,8019,It 's one of the most honest films ever made a...,1,s honest film hollywood
29337,453,An absorbing and unsettling psychological drama .,1,absorb unsettling psychological drama
29338,13097,"Soylent Green IS...a really good movie, actual...",1,soylent green good movie actually ve think don...
29339,26896,There just isn't enough here. There a few funn...,0,isn funny spot disappoint love stupid movie ex...


In [18]:
%%time
processed_text = []

for text in tqdm(imdb_test_data['review']):
    tmp = Text_processing_pipeline(text).main_pipeline()
    processed_text.append(tmp)

100%|██████████| 29341/29341 [19:50<00:00, 24.65it/s]CPU times: user 19min 50s, sys: 2.13 s, total: 19min 52s
Wall time: 19min 50s



In [20]:
# imdb_test_data.to_csv('./data/processed_test.csv',index=0)
imdb_test_data['processed_review'] = processed_text

,ID,review,processed_review
0,22622,Robert Lansing plays a scientist experimenting...,robert lansing play scientist experiment pass ...
1,10162,"Well I've enjoy this movie, even though someti...",ve enjoy movie turn stereotypical situation nt...
2,17468,First things first - though I believe Joel Sch...,thing believe joel schumacher well mediocre di...
3,42579,I watched this movie on the grounds that Amber...,watch movie ground amber benson rock nick stah...
4,701,A certain sexiness underlines even the dullest...,certain sexiness underline dull tangent
...,...,...,...
29336,30370,It is difficult to rate a writer/director's fi...,difficult rate writer director s effort movie ...
29337,18654,"After watching this movie once, it quickly bec...",watch movie quickly favorite different event h...
29338,47985,"Even though i sat and watched the whole thing,...",sit watch thing good place big chunk informati...
29339,9866,Warning Spoilers following. Superb recreation ...,warn spoiler follow superb recreation base ant...
